### Friday, December 8, 2023

[TheBloke/Orca-2-13B-AWQ](https://huggingface.co/TheBloke/Orca-2-13B-AWQ)

docker container start hfpt_Nov20

To load this model I had to run ... 'pip install autoawq'

Hmm getting a ' libcudart.so.12: cannot open shared object file: No such file or directory
' error ... reading a fix is to run ..

pip uninstall torch torchvision torchaudio triton

then run ...

pip install torch torchvision torchaudio

OK Nice! I am now able to step through this notebook!

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
!pip show transformers

Name: transformers
Version: 4.36.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.8/dist-packages
Requires: packaging, regex, huggingface-hub, tqdm, filelock, numpy, requests, tokenizers, pyyaml, safetensors
Required-by: lm-eval, autoawq


In [4]:
!ls /root/.cache/huggingface/hub/

models--Salesforce--blip-image-captioning-large  version.txt
models--TheBloke--Orca-2-13B-AWQ


In [5]:
# Load in the "TheBloke/Orca-2-13B-AWQ" model
# docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--Orca-2-13B-AWQ 9ef7ecd7959f:/root/.cache/huggingface/hub
# Successfully copied 7.25GB to 9ef7ecd7959f:/root/.cache/huggingface/hub

In [6]:
# !ls /home/rob/Data2/huggingface/transformers
!ls /root/.cache/huggingface/hub/

models--Salesforce--blip-image-captioning-large  version.txt
models--TheBloke--Orca-2-13B-AWQ


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

In [8]:
model_name_or_path = "TheBloke/Orca-2-13B-AWQ"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [10]:
%%time

# Nice! This loads to the GPU! 7530 MiB
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, low_cpu_mem_usage=True, device_map="cuda:0")

# this too takes up CPU RAM
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", trust_remote_code=False, revision="main")

# model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", trust_remote_code=False)

# Download time ...
# 103m 59.2s

CPU times: user 4.07 s, sys: 1.31 s, total: 5.38 s
Wall time: 4.07 s


In [12]:
# Using the text streamer to stream output one token at a time
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [11]:
# This system message was copied from ... https://huggingface.co/microsoft/Orca-2-13b
system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."

In [13]:
prompt = "Tell me about AI"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''

In [14]:
# Convert prompt to tokens
tokens = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

In [15]:
generation_params = {
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.95,
    "top_k": 40,
    "max_new_tokens": 512,
    "repetition_penalty": 1.1
}

In [16]:
# Generate streamed output, visible one token at a time
generation_output = model.generate(
    tokens,
    streamer=streamer,
    **generation_params
)

AI stands for artificial intelligence, which is the field of computer science that studies how to create machines or systems that can perform tasks that normally require human intelligence, such as reasoning, learning, decision making, perception, natural language processing, etc. AI can be divided into two main branches: machine learning and artificial neural networks. Machine learning is the process of designing algorithms and data structures that can learn from experience and improve their performance on a given task. Artificial neural networks are models inspired by the structure and function of biological neurons that can process and transmit information using mathematical operations. AI has many applications in various domains, such as healthcare, education, entertainment, security, finance, manufacturing, transportation, etc. AI can also pose ethical, social, and legal challenges that need to be addressed by researchers, policymakers, and society at large.<|im_end|>


In [17]:
# Generation without a streamer, which will include the prompt in the output
generation_output = model.generate(
    tokens,
    **generation_params
)


In [18]:
# Get the tokens from the output, decode them, print them
token_output = generation_output[0]
text_output = tokenizer.decode(token_output)
print("model.generate output: ", text_output)

model.generate output:  <s><|im_start|>system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.<|im_end|>
<|im_start|>user
Tell me about AI<|im_end|>
<|im_start|>assistant
AI stands for artificial intelligence, which is the field of computer science that studies how to create machines or systems that can perform tasks that normally require human intelligence, such as reasoning, learning, decision making, perception, natural language processing, etc. AI can be divided into two main branches: narrow AI and general AI. Narrow AI refers to systems that can perform specific tasks or domains, such as face recognition, speech synthesis, chess playing, etc. General AI refers to systems that can exhibit human-like intelligence across a wide range of domains, such as understanding natural language, planning, creativity, etc. General

In [19]:
# Inference is also possible via Transformers' pipeline
from transformers import pipeline

In [20]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    **generation_params
)

In [21]:
pipe_output = pipe(prompt_template)[0]['generated_text']
print("pipeline output: ", pipe_output)

pipeline output:  <|im_start|>system
You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior.<|im_end|>
<|im_start|>user
Tell me about AI<|im_end|>
<|im_start|>assistant
AI stands for artificial intelligence, which is the field of computer science that studies how to create machines or systems that can perform tasks that normally require human intelligence, such as reasoning, learning, perception, decision making, and natural language processing.<|im_end|>
